In [ ]:
import torch
from transformers import VitsModel, AutoTokenizer
import scipy.io.wavfile
from IPython.display import Audio, display


MODEL_ID = "facebook/mms-tts-vie"
OUTPUT_FILE = "output_mms_vie.wav"
TEXT_INPUT = "Xin chào, đây là thử nghiệm mô hình VieNeu TTS trên Google Colab. Tôi là dân kỹ thuật về lĩnh vực MLOps."

def run_mms_tts(text, model_id, output_path):
    print(f"tải model: {model_id}...")

    # Load model & tokenizer
    model = VitsModel.from_pretrained(model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    print("xử lý text sang waveform")
    inputs = tokenizer(text, return_tensors="pt")

    # Inference (không cần tính gradient)
    with torch.no_grad():
        output = model(**inputs).waveform

    # MMS output là float tensor, cần chuẩn hóa để lưu file wav
    # Dữ liệu thường nằm trong khoảng [-1, 1], output shape: (1, num_samples)
    waveform = output.float().numpy().squeeze()

    # Lưu file
    print(f"lưu file: {output_path}")
    scipy.io.wavfile.write(output_path, rate=model.config.sampling_rate, data=waveform)

    return output_path


try:
    wav_path = run_mms_tts(TEXT_INPUT, MODEL_ID, OUTPUT_FILE)
    print("Xong!")
    display(Audio(wav_path, autoplay=True))
except Exception as e:
    print(f"lỗi xảy ra: {e}")